# Tutorial 4 - Contacts and Conditioning
**Aim:** To finally make a friggin' leg! Aw yisss. But also to properly confront the problem of hard contacts, and what we do about them.

**Further reading:** There are two important papers I'll be referring to, but I'd suggest rather reading them afterwards:
1. <a href="https://jeb.biologists.org/content/jexbio/202/23/3325.full.pdf"> Templates and Anchors: Neuromechanical Hypotheses of Legged Locomotion on Land - RJ Full and DE Koditschek (1999) </a>
2. <a href=https://journals.sagepub.com/doi/pdf/10.1177/0278364913506757> A direct method for trajectory optimization of rigid bodies through contact - M Posa, C Cantu and R Tedrake (2014) </a>

If you have a bit more time and want to immerse yourself in one of the religious texts of legged robotics, check out *Legged Robots that Balance* by Marc Raibert. (The dude founded Boston Dynamics, so I reckon he knows what he's talking about.)  

## Templates and Anchors
The model we'll be constructing consists of one leg (two links connected by a prismatic joint) and a body:

<img src = "hopper.png" width = "400">

A monopedal hopper probably isn't the most useful configuration in real life, but it's still an important model that helps biologists and roboticists alike to understand legged locomotion.

Considering that most of the macroscopic, multicellular life forms on Earth are insects, the average number of legs that an animal has is... in my estimation... more than one. So you may be wondering how a one-legged model can tell us anything useful about any real critter. But imagine somebody told you to come up with the shortest possible "executive summary" of the running motion of, say, a cheetah. (It's always a cheetah... ;P) The first thing you'd do is remove all the non-essential bits: so head, flexy torso and tail become just one rigid body. Or maybe even a point mass. And then you notice that only one leg is ever really in contact with the ground at a time, and hey, even if more than one was in contact, there is only one centre of pressure, so why do we even need more than one 'leg?\*'... And soon enough, you're left with a hopper. The outcome would be the same if you tried it for a human, or a cockroach, or pretty much any other legged animal you could think of (except maybe a millipede... I suspect locomotion fundamentals change when the number of legs hits double figures...) 

\* this single "leg" (well, limb betweent the body and centre of pressure) used to represent the action of more than one leg is referred to as a *virtual leg.*

In biomechanics, this "executive summary" model for a motion - i.e. the lowest degree-of-freedom model that still captures its essential characteristics - is called a *template*. More complex models are called *anchors*, since they anchor the template behaviour into the more realistic, detailed behaviour of the system. In addition to the obvious simplification benefit, templates are beneficial because they allow us to compare the motion of systems with widely differing morphologies.  

...So that's why we care about monopeds: from Raibert's bouncing robots to the spring-loaded inverted pendulum (SLIP) models frequently applied to represent running, it's clear they are a good template for legged locomotion. 

**The takeaway:** Rather than just giving you some cool biomechanical jargon to throw around, the thing I want you to get out of this template idea is that dynamic modelling should be a modular process. Trajectory optimization is extremely computationally demanding, so we have to be as economical as possible with our degrees of freedom. Being efficient with your coordinates can make the difference between finding a solution in 15 minutes, or three hours.

Of course, that doesn't mean we should never use high-order models: anchors are absolutely essential to understanding motion. It just means you should get the simplest model working first, and then build complexity slowly and iteratively from there.

In [ ]:
%reset 
# clears variables in workspace
# note on resetting:
# especially while you're debugging, you want to clear your variables between attempts to solve. Why?
# A) you may have noticed that pyomo doesn't like things being redefined
# more importantly, B) the variables keep their values, so an infeasible solution becomes an infeasible guess. 

# import libraries
# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

from IPython.display import display #for pretty printing

# create the model
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

#N = 100
N = 100
m.N = RangeSet(N) 

# sets can have multidimensional indices
# (probably a little gratuitous for such a simple model, but thought I'd show you anyway)
links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta']
m.DOF = Set(initialize = DOFs) # the coordinates for each link

***Annoying sidenote on the subject of set creation**

When you create a set, always do it like this:

```python
SpiceGirls = ['ginger','baby','posh','scary','sporty']
m.sg = Set(initialize = SpiceGirls)
```
or perhaps like this:

```python
m.sg = Set(initialize = ['ginger','baby','posh','scary','sporty'])
```
(though the first one is nicer, because having a list of the names it makes it easier to iterate over members of that set later if you have to.)

But **never** like this:

```python
m.sg = Set(['ginger','baby','posh','scary','sporty'])
```
Daft as it is, that last one actually creates five empty *sets*, instead of what you want: a single set with five values.

You can probably get away with it in many circumstances, but attempting to index using a set obviously makes absolutely no sense, so you're going to run into an error sooner or later. (If you ever come across the error **"cannot index a component with an indexed set"**, this may be why.

In [ ]:
# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

# whenever a multidimensional set is passed to a function, the index is expanded (infuriatingly enough...)
# so every function indexed on L needs to expect two inputs from that set 
# I've called them l = [lb, ln] for 'branch' and 'number'

def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.X = Var(m.N, m.L, m.DOF) # position
m.dX = Var(m.N, m.L, m.DOF) # velocity
m.ddX = Var(m.N, m.L, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    for l in links:
        m.X[n,l,'y'].setlb(0.0)

## Contacts
The problems at the fixed point in the last tutorial started to reveal some of the difficulties around formulating hard boundaries in an optimizaion problem. This tutorial is going to elaborate on those issues and explain the workarounds we use to overcome them. Well, maybe "overcome" is a strong word... often the best we can do is just making the situation less dire so it can actually solve within a human lifetime and only break for half the random seeds we throw at it instead of most of them.

I'll get more specific as we go, but as a general overview, contacts are difficult to deal with for two primary reasons:

**They're discontinuous.** Imagine trying to draw a freebody diagram representing the forces acting on a four-legged animal: you'd have the weight, the torques acting at each of its joints, the normal force on each of its paws... But only if it actually happened to be standing on all four paws. It could have just one foot on the ground, or two, or three, or *the other three*, or none at all or... you get the point. Changing the contact state essentially changes the dynamic model you're working with by switching different forces 'off' or 'on'. 

There is an approach called *hybrid modelling* that makes this explicit: each contact state gets its own dedicated dynamic model with switches between models happening at specified events, such as foot touch-downs and lift-offs. But this isn't suitable for our needs because:
1. The number of discrete models is prone to explode. For a biped, you need four: right foot down, left foot down, neither, both. For a quadruped, you need sixteen. And that's just assuming the foot sticks in place and doesn't slide. 
2. You need to know the contact order up front. Not a problem if you're modelling a well-established motion pattern like a gallop, but our research questions are often around less-defined manoeuvres like rapid stops or turns, or involve finding out the best movement to achieve something, and therefore requires that the model isn't limited to a specific contact order.

...So we need a single model that can apply the right combination of forces under the right conditions. If I had to ask you to describe such a thing using pseudocode or a flow chart, it would probably involve a lot of IF statements:

```python
if right_foot_contact:
    right_GRF_active = True
    ```
...that sort of thing. Problem is, NLP solvers are always (at least, to my knowledge) gradient-based to some extent, and like nice, smooth derivatives they can slide down to an answer. Conditionals like that create sharp cliffs that they absolutely can't deal with. To get around this, we cheat using something called *complementarity*... but we'll get to that just now.

**They're fast.** Which brings me to... 

### Contacts and timing
Hard boundaries themselves aren't really the problem - it's actually the dynamics of collisions that's tricky. Hard boundaries merely create a site where collisions must occur.

Collisions are hard because they're *impulsive events:* in continuous time world, their dynamics involve very large forces that act over infinitesimally short time intervals. But in trajectory optimization world, there's a finite minimum timestep over which forces must act. 

Another way to think about it, if you'll allow me a moment to put my Signals and Systems I TA hat on, is that we're attempting to deal with high frequency signals within the confines of a slow sample rate. An IF statement is just a step function, and think of all the high-frequency harmonics you need to make a step look truly *crispy*... Now imagine what it would look like aliased to the 50 - 100 Hz sample rate we typically limit ourselves to. Nyquist is shaking.

So... why not just sample faster? To quote an ancient meme proverb: "Ain't nobody got time for that." Shorter timestep = more nodes to cover the same duration = more variables and constraints = huge problem size = centuries of solving time.

Ideally, what you want is a slow sample rate for the continuous parts and a fast sample rate for when collisions happen. And that's exactly what we do! Within limitations. 

But let's start with the good news: a variable timestep allows the flexible sampling rate we desire, and also allows collisions to occur more naturally. See, the model can only change states at the boundaries of a timestep, meaning that if you're using a fixed step of say 0.01 seconds, the foot can only touch down or the joint hit its limit or whatever at $t=$ some precise multiple of 0.01. Putting some bend in the timestep theoretically allows it to stay in a state for an arbitrary duration.

Unfortunately, there is only so much bend we can allow. Introducing a variable timestep makes the problem even more nonlinear, and we have been warned by Prof Larry Biegler (the First of his Name, High Archmage of trajectory optimization, Coder of IPOPT) that allowing it too much flexibility can cancel out the benefits and make the problem harder to solve. Based on his rule of thumb, we therefore allow it to vary only within 20% of a master timestep.
_______
## The scaling problem
If you do any reading on the topic of optimization, you'll probably hear about the *conditioning* of problems. A *well-conditioned* optimization problem is something an NLP solver is well-equipped to deal with: we've already mentioned that solvers like smooth, convex functions. Another thing they really like is a *well-scaled* problem: that is, one where all the variables and constraint derivatives are a similar order of magnitude.

Alas, we find ourselves in an especially difficult predicament here. If you had to describe the worst possible system to do trajectory optimization on... The 9th Circle of Badly-Conditioned Hell... whatever you come up with would probably be pretty close to the systems we work with.

Consider the velocity integration constraint. It includes:
* velocities with magnitudes in 1's or 10's
* accelerations that might have magnitudes similar to the velocities, but can also have magitudes in the 100's, 1000's or even 10 000's when collisions occur
* our now-variable timestep in the order 0.01

In combination with the horribly infeasible starting points our random initialization method creates, it's not surprising that these problems take ages to solve, frequently come back infeasible, or crash the algorithm entirely. Even once your model has passed a bunch of sanity checks and been debugged to the extent that you're confident all the equations, etc. are fundamentally correct, you might still hit the dreaded *restoration failed* error. I don't want to get too in-depth into the technical details of IPOPT, but when you see that, what's happened is the optimization gnome tasked with solving your problem has had a breakdown, given up, flipped his desk and stormed out, viciously cursing you, your robot, your ancestors, and the entire science of mathematics all the while.

### Scaling - an overview
One of the things we can do to make our poor gnome's life easier is attempt to scale our problem better. The inherent properties of our system mean its probably never going to be well-scaled, but a bit of tinkering can make it slightly less than a complete trash fire.

One of my few complaints about Pyomo is that it doesn't have an inbuilt scaling feature, meaning we have to introduce scale factors manually. This can be a good thing, since it's much more flexible, and you know exactly how the scaling is implemented, but it does introduce the potential for human error so you need to keep your wits about you.

Scaling is yet another one of those *art* things (I'm sure you're getting really tired of those by now) so I don't really have a precise, step-by-step theory of scaling you can follow. The best I can do is tell you the general guidelines and explain a few specific cases as I go along. 

Here's what we're trying to achieve by scaling:
1. the values of all the variables in the solution should be around 1 ("around 1" here meaning order of magnitude 1 ideally, but somewhere between 0.01 and 100 pragmatically.)
2. The values of all constraint derivatives should be around 1. For linear constraints, this is as simple as making sure all the coefficients are within the established "around 1" range. For nonlinear constraints, it's much trickier, since the "coefficients" (when you think of the partial derivative in a particular variable) become functions of the variables. 
3. The value of each term in a constraint equation should be around 1.

With our problems, it's almost impossible to satisfy all of these at once. You might have to make trade-offs. For variables with wide ranges, work with the mean values you expect them to take on and accept that the nodes where the extreme values occur will be nasty. Most of your variables shouldn't need scaling: start with the most egregious cases (e.g. the timestep or the contact forces, which tend to have magnitudes furthest outside these bounds) and go from there.  

This is all complicated further by the inescapable tangle of interdependency: how well your model works is tightly tied up with the initial guess, the timestep and the duration of the simulation. Finding the right balance of all these things is the process of *conditioning* your model a.k.a. all the kerfuffle that comes after writing and basic sanity checks, but before the model is ready to be rolled out for your experiment. Conditioning is the worst, most rage-inducing part of trajectory optimization, and often takes far longer than actually writing the model. My approach to it is to try to separate each of the aspects into a separate step:
1. **scaling:** I choose whichever scale factors seem logical based on the stated guidelines, adding them iteratively and keeping those that don't appear to actively worsen the performance. Testing the scaling on your target problem can be tricky, since it might not solve at all until you've tweaked the timing, so use a sanity check you know it can pass already. (I provide a few examples at the end of this notebook). The `m.pprint()` readout is super helpful for pinpointing the variables and equations that need the most work.


2. **timing:** Now you're ready to try your target problem. You probably have some idea of how long the motion should take, so start with that. If it doesn't solve, add or remove nodes based on whether $h$ is tending to hit the upper or lower bound. (Technically, you could change the duration by changing the timestep, but I find that causes scaling issues, so I prefer to keep $h_m$ somewhere between 0.01 and 0.02 seconds and adjust the duration using the number of nodes instead.)


3. **initialization:** This is where the model gets most temperamental, so I wouldn't recommend even touching this step until you have a solution converged with the default initial values. Before wasting time here, you need to know that the motion is achievable and how long it takes. Remember: unless you're *warm-starting* from a previous solution, whatever you add here is likely to be less feasible than the default input - if it can't solve from the default point, it definitely can't solve from whatever weird point you inflict on it. 

You could really keep fiddling forever, so it's important to know when to stop: you're not going for *perfect*, or even *good* - you're going for *usable*.

### Scaling example 1: velocity integration
We want a master timestep $h_m = 0.02$, meaning the variable timestep has a range $0.016 \leq h[n] \leq 0.024$. We could just create an $h$ variable with those as the bounds, but it fits much better into the Around 1 Zone if we think of the timestep as $h_m h[n]$, since the variable part then has the range $0.8 \leq h[n] \leq 1.2$.

This means that $h_m$ becomes our scaling factor for $h$: wherever we use it in an equation, it will be as the product of the scalar part and the variable part. The integration constraint therefore becomes: $$\dot{x}[n] = \dot{x}[n-1] + 0.02h[n]*\ddot{x}[n]$$

The typical velocity and acceleration values should be around 10, maybe around 100 for the latter, so the partial derivatives and values of individual terms should come out Around 1 ($10 = 10 + 0.02 \times 1 \times 10$). Even if we allow for the more extreme accelerations, the value of that tricky last term should still be on the high side of Around 1 ($0.02 \times 1 \times 10000$). That's probably good enough.

In [ ]:
# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,lb,ln,dof): # for positions
    l = (lb,ln) # why. whyyyyyy.
    if n > 1:
        return m.X[n,l,dof] == m.X[n-1,l,dof] + hm*m.h[n]*m.dX[n,l,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.L, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,lb,ln,dof): # for velocities
    l = (lb,ln) #ffs
    if n > 1:
        return m.dX[n,l,dof] == m.dX[n-1,l,dof] + hm*m.h[n]*m.ddX[n-1,l,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.L, m.DOF, rule = BwEuler_v)

## Complementarity
Complementarity is how we finesse conditional behaviour out of functions that are still somewhat compatible with an NLP solver. *Complementing* two quantities just means specifying that their product is zero. This effectively creates a NAND relationship between them: the constraint is satisfied if one or both of the variables is zero, but FALSE if they're both nonzero at the same time. For our purposes, the complemented quantities **must be positive.**

In an ideal world, you'd simply be able to write $AB = 0$ and be done with it, but as you've seen already, trajectory optimization world is far from ideal. Constraints of this form are actually extremely difficult for most NLP solvers to deal with. IPOPT in particular is an interior point solver, which means it looks for the feasible interior region between the multidimensional surfaces formed by the constraints. Complementarity constraints don't have an interior, though: the solution lies along the axis of one complemented variable or the other, since the value of at least one of them must be squashed down to zero. To improve IPOPT's chances, we create a false interior by setting $AB = p$, where $p$ is a penalty variable we subsequently minimize as part of our cost function. 

Throughout the rest of this tut, you'll see how we construct complementarity constraints to deal with assorted hard contact problems.

## Ground interactions

The ground reaction force has two components:
1. the vertical normal force $G_y$. This can only act upwards, so $G_y \geq 0$.
2. the horizontal friction force $G_x$. This can act in either direction, but we represent it using two positive variables $G_x^+ > 0$ and $G_x^- > 0$ such that $G_x = G_x^+ - G_x^-$. Breaking a variable into positive and negative components is something you'll see happening all over the place to facilitate complementarity. Most of the time, it's not necessary to complement these variables with each other, but occassionally you might need to do so to ensure that they can't make weird things happen by being nonzero at the same time. A few back-of-the-envelope calculations never go amiss.

We use four complementarity constraints to bring about this behaviour:

### Contact
**Purpose:** Makes sure $G_y$ only acts when the foot is on the ground.

**Constraint:** $G_y[n]y_{foot}[n+1] = p[n]$

We like complementarity equations to be as simple as possible, so we use an auxiliary variable $y_{foot}$ to represent the foot height. Note that the normal force at *this* timestep is complemented with the foot height at the *next* timestep. Sure this means that the force technically starts acting one node before the foot is directly in contact with the ground, but this is necessary to prevent problems of the kind we saw with the fixed point pendulum: if you want the foot to hit the ground and stay down, it means that the normal force acting on the touchdown node - where the foot has a height of zero, but still has its downward velocity - has to be *exactly* the force required to negate that velocity in one step. If it can't be (and we established that, due to the maths resulting from the link end position not being the dynamic variable, it probably won't be) it has to be inaccurate in the upward direction, since the foot absolutely can't penetrate the floor. Without that timestep's difference, you'd see horrible bouncing as the foot is pushed up off the ground.

### Friction
**Purpose:** Only allows the foot to have a horizontal velocity (that is, to slide) when it maxes out the static friction limit.

**Constraint:** $(\dot{x}_{foot}^+[n] + \dot{x}_{foot}^-[n])f[n] = p[n]$

Notice that we've split the foot velocity into positive and negative components, so their sum indicates whether the magnitude is nonzero. 

The aux variable $f$ represents what we call the *friction cone*, though in 2D, it's more like the friction triangle: 

<img src = "friction_cone.png" width = "300">

if the magnitude of the velocity is inside the cone, the foot sticks. If it hits the edge of the cone, the foot slides. Another way of putting it: if the foot is moving while in contact with the ground, the friction force must be at its maximum value $\mu G_y$. The difference between the static friction limit and the friction's actual magnitude is what we define as $f$: $f = \mu G_y - (G_x^+[n] + G_x^-[n])$.

### Sliding

**Purpose:** Makes the friction force act in the opposite direction to the foot's velocity.

**Constraint:** This one has two parts - one for each direction: $\dot{x}_{foot}^+[n]G_x^+[n] = p[n]$ and $\dot{x}_{foot}^-[n]G_x^-[n] = p[n]$
_____
### Scaling example 2: contact forces and penalties

I scale all the forces\* in my system by the body weight ($BW$), but you'll notice that I don't include this scale factor in every single equation where these forces appear. Take the contact complementarity constraint: $$p_{contact}[n] = G_y[n] y_{foot}[n]$$

In this constraint, the magnitude of the force doesn't have any physical importance - it only matters whether it's zero or not. In the equations of motion, where its value does matter because it's being used to calculate the acceleration, I use $BW G_y$. The whole point is for the variable part to end up with a typical magnitude of 0.1 to 1, rather than 10 to 100. The foot height is also likely to be in this range, so if the factor $BW$ was included in the complementarity equation, the scaling would suddenly become unbalanced (i.e. the partial derivative with respect to $y_{foot}$ would be an order or two larger than the partial with respect to $G_y$.)

What about the penalty? We intend for it to be zero, but we know realistically we might be able to get it down to, say, $< 1e^{-6}$ at best. So does that mean you should scale it by some teeny tiny value? Based on my experience, I'm inclined to say no. (I wish I had some robust mathematical reasoning to throw at you, but my best guess is that including that little scale factor as a coefficient in the equation throws the partials off-balance.)
_______
\* I also scale the torques by $BW$, but they are a saga all on their own that I'll talk about in the context of the equations of motion rather.

In [ ]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

# variables
# (I had that set error while trying to set up these default bounds. That's why I pointed it out.)

m.footp = Var(m.N, m.DOF, bounds = (0.0,None)) # foot position
m.footv = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone = Var(m.N, bounds = (0.0,None))

m.GRF = Var(m.N, m.DOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# constraints: aux variables

def def_footp(m,n,dof):
    if dof == 'y':
        return m.footp[n,dof] == m.X[n,('leg',2),'y'] - 0.5*m.len[('leg',2)]*cos(m.X[n,('leg',2),'theta'])
    else:
        return Constraint.Skip
m.def_footp = Constraint(m.N, m.DOF, rule = def_footp)

def def_footv(m,n,dof):
    if dof == 'x':
        return m.footv[n,dof,'ps']-m.footv[n,dof,'ng'] == m.dX[n,('leg',2),'x'] + m.dX[n,('leg',2),'theta']*0.5*m.len[('leg',2)]*cos(m.X[n,('leg',2),'theta'])
    else:
        return Constraint.Skip
m.def_footv = Constraint(m.N, m.DOF, rule = def_footv)

def def_friction_cone(m,n):
    return m.friction_cone[n] == m.mu*m.GRF[n,'y','ps'] - (m.GRF[n,'x','ps'] + m.GRF[n,'x','ng'])
m.def_friction_cone = Constraint(m.N, rule = def_friction_cone)

# constraints: complementarity

# contact
def ground_contact(m,n):
    if n < N:
        return m.ground_penalty[n,'contact'] == m.footp[n+1,'y']*m.GRF[n,'y','ps'] 
        # notice that the GRF is complemented with the foot height at the NEXT node
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, rule = ground_contact)

# friction
def ground_friction(m,n):
    return m.ground_penalty[n,'friction'] == (m.footv[n,'x','ps']+m.footv[n,'x','ng'])*m.friction_cone[n]
m.ground_friction = Constraint(m.N, rule = ground_friction)

# slipping
def ground_slip_ps(m,n):
    return m.ground_penalty[n,'slip_ps'] == m.footv[n,'x','ps']*m.GRF[n,'x','ps']
m.ground_slip_ps = Constraint(m.N, rule = ground_slip_ps)

def ground_slip_ng(m,n):
    return m.ground_penalty[n,'slip_ng'] == m.footv[n,'x','ng']*m.GRF[n,'x','ng']
m.ground_slip_ng = Constraint(m.N, rule = ground_slip_ng)

# bound contact forces at first node
for dof in DOFs:
    for sgn in signs:
        m.GRF[N,dof,sgn].value = 0
        m.GRF[N,dof,sgn].fixed = True

## Joints with hard end stops

In the last tutorial, I suggested that you visualize the fixed point as a bolt sliding in an increasingly narrow slot to understand the problem. We're essentially dealing with a less extreme version of the same thing when we model a joint with a limited range of motion.

You may wonder why we need to go to all this trouble: after all, if we wanted to get a solution out where the hip angle doesn't exceed $\frac{\pi}{2}$, all we need to do is set the upper limit on its variable to $\frac{\pi}{2}$, right?

And sure, that approach might give you a solution, if you're very lucky, but in all likelihood, that solution will be a bit kak. Remember, the solver doesn't know it's dealing with a hopper, or a pendulum, or a robot or whatever you're trying to model; it has a list of variables, their relationships to each other and the acceptable ranges of values they're allowed to fall into. If we introduce a limitation on the motion of the system, we also have to give the solver the means to fulfil that limitation in the form of the forces or torques that would act in the real-life situation. (This is also why we need to provide a constraint action for every relative degree of freedom removed by a joint.) Otherwise, you're restricting the solution space to just those motions where the angle of the leg just happens to swing less than the given boundary.

In an actuated joint, the problem is less obvious since the actuator can perform the role of the rebound force to some extent. This is why, if you don't bother to include complementarity-controlled rebound forces, your actuator profiles will be a heinous barcodey mess, since the poor unfortunate motor will be forced to reverse direction to slow the link down before it exceeds the joint limit. Naturally, this also has implications for the resulting motion: without the possibility of encountering a stopping impulse, the limbs will always have to decelerate before they hit their limits - they won't be able to swing full tilt right to the end of the ROM.

### Hard stops in a rotary joint
Hard joint stops work exactly like the contact constraint in the ground block, only now, instead of the foot height, it's the distance to the endpoint, and instead of the ground reaction force, it's a rebound action opposing motion beyond the limit.

Only two complementarity constraints are needed - one for each boundary:

On the upper side: $\tau_r^-[n](\theta_{max} - \theta[n+1]) = p[n]$

And on the lower side: $\tau_r^+[n](\theta[n+1] - \theta_{min}) = p[n]$

In [ ]:
# ROTARY JOINT --------------------------------------------------------------------------------------------------------------
# the model's "hip"
# points of interaction: 1) the body COM 2) the top of leg link 1

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2,np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 

# variables
m.top = Var(m.N, m.DOF) # we only need the top of leg link 1

m.th_joint = Var(m.N, bounds = (hip_bound[0],hip_bound[1])) 

m.tau_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque

tau_lim = 2
m.tau_a = Var(m.N, bounds = (-tau_lim,tau_lim)) # actuator torque

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

m.F_c = Var(m.N, m.DOF) # constraint forces 

# constraints

# aux variables
def def_top(m,n,dof):
    if dof == 'x':
        return m.top[n,dof] == m.X[n,('leg',1),'x'] - 0.5*m.len[('leg',1)]*sin(m.X[n,('leg',1),'theta'])
    if dof == 'y':
        return m.top[n,dof] == m.X[n,('leg',1),'y'] + 0.5*m.len[('leg',1)]*cos(m.X[n,('leg',1),'theta'])
    else:
        return Constraint.Skip
m.def_top = Constraint(m.N, m.DOF, rule = def_top)
    
def def_th_joint(m,n):
    return m.th_joint[n] == m.X[n,('leg',1),'theta'] - m.X[n,('body',1),'theta']
m.def_th_joint = Constraint(m.N, rule = def_th_joint)

# connection
def connect_hip(m,n,dof):
    if dof == 'theta': 
        return Constraint.Skip
    else:
        return m.top[n,dof] == m.X[n,('body',1),dof]
m.connect_hip = Constraint(m.N, m.DOF, rule = connect_hip)

# complementarity
def hip_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.th_joint[n+1])*m.tau_r[n,'ng']
        else:
            return m.joint_penalty[n,'hip',jc] == (m.th_joint[n+1] - m.hip_bound['lo'])*m.tau_r[n,'ps']
    else:
        return Constraint.Skip
m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

# bound contact forces at last node
for sgn in signs:
    m.tau_r[N,sgn].value = 0
    m.tau_r[N,sgn].fixed = True

## Creating a prismatic joint
A prismatic joint allows relative translation between the points of interaction (in this case, the COMs of the two leg links) but not relative rotation. 

<img src = "prismatic.png" width = 300>

The connection constraints must therefore:
1. force the links to have the same angle, and
2. make sure their COMs are colinear (so you don't end up with two parallel links just hanging around in random places in space.)

The constraint action in this case is a torque, $\tau_c$.

I find it easiest to think about the actuator and rebound forces acting parallel to the leg, and only breaking them into their $x$ and $y$ components when I incorporate them into the EOMs. The direction convention I use is positive expands the joint, and negative contracts it. (Note: be careful when you put them into your equations of motion - the links will be pushed in opposite directions.)

In [ ]:
# PRISMATIC JOINT -----------------------------------------------------------------------------------------------------------
# the model's "knee"
# points of interaction: 1) leg link 1 COM 2) leg link 2 COM

# parameters
knee_bound = [0.0,0.5]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]}) 

# variables
m.r_joint = Var(m.N, bounds = (knee_bound[0],knee_bound[1])) # the length of the joint

m.F_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound force (acts parallel to the leg)

m.F_a = Var(m.N, bounds = (-3,3)) # actuator force (also parallel)

m.tau_c = Var(m.N) # constraint torque

# constraints

# connection
def connect_knee(m,n,dof):
    # colinearity
    if dof == 'x':
        return m.X[n,('leg',2),dof] == m.X[n,('leg',1),dof] + m.r_joint[n]*sin(m.X[n,('leg',1),'theta'])
    if dof == 'y':
        return m.X[n,('leg',2),dof] == m.X[n,('leg',1),dof] - m.r_joint[n]*cos(m.X[n,('leg',1),'theta'])
    # rotation
    if dof == 'theta':
        return m.X[n,('leg',2),dof] == m.X[n,('leg',1),dof]
m.connect_knee = Constraint(m.N, m.DOF, rule = connect_knee)

# complementarity
def knee_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT distance
            return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] - m.r_joint[n+1])*m.F_r[n,'ng']
        else:
            return m.joint_penalty[n,'knee',jc] == (m.r_joint[n+1] - m.knee_bound['lo'])*m.F_r[n,'ps']
    else:
        return Constraint.Skip
m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# bound contact forces at first node
for sgn in signs:
    m.F_r[N,sgn].value = 0
    m.F_r[N,sgn].fixed = True

## Minimizing the penalty
Another tip from Prof Biegler is to force priority on the penalty by magnifying it with some large scale factor in the cost function. There isn't really any exact science for calculating the right value - I typically go with 1000, or whatever amounts to three orders of magnitude above the thing I'm actually trying to minimize. Your mileage may vary.

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------

# minimum time

def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for jc in joint_constraints:
            for j in joints:
                penalty_sum += m.joint_penalty[n,j,jc]
    return T+1000*penalty_sum
m.Cost = Objective(rule = CostFun)


### Scaling example 3: scaling equations
You don't have to limit your scaling to the variables - you can also scale equations by multiplying both sides by the same factor. For instance, if you had an equation $1000z = 5000x + 2000y$, you could rewrite it $z = 5x + 2y$. The relationship between the variables wouldn't change, but you'd prevent those huge coefficients going into the constraint matrix.

Since all the forces are scaled by the body weight, the equations of motion have a form $m\ddot{x} = BW F$, and therefore we can get all the coefficients closer to Around 1 by dividing everything by $m$.

**Scaling example 3B: the one where everything is terrible and I have no idea what I'm doing**

Even compared to the baseline awfulness of the rest of the problem, scaling the torques and $\theta$ equations is a nightmare. Here is a brief run-down of why they suck more than anything else has ever sucked:

* Moments of inertia scale quadratically. Hence, even though the leg links in this problem are just half the length and weight of the body link, their moments are a full order of magnitude less. 
* Further, the momet of inertia of a link can be one or two orders of magnitude smaller than its mass. 
* Any forces that don't act parallel to a link or through its COM create moments, so these tiny inertia values end up in the same equation as variables scaled to the body weight. (And yes, that is taking the distance from the COM into account... you still end up with coefficients in the 0.01's range plonked into the same equations as coefficients in the mid-high 10's range. And don't even get me started on the fact that some terms are related to sin and cos functions of the angles.)
* The torques applied over a trajectory vary widely: depending on whether the foot is on the ground or not, the torque required to swing the leg to a particular velocity or constrain a joint can be fractions or hundreds of Newton.

I don't know what the best approach is here. There are many things that might seem logical to try, and the only advice I can give is to throw as many of them at the wall as possible and see what sticks. For example, I tried scaling the torques by the body weight, scaling them all by the body's inertia, scaling each one by the larger inertia out of the two links it acts on, dividing the $\theta$ EOM's by the links' inertias, losing all faith in my own understanding and quitting engineering to become a professional cat petter, screaming my wordless despair at the sky, performing ancient black magick rituals and ultimately begging for the sweet release of death before deciding that the body weight thing was the least worst.

In [ ]:
# EQUATIONS OF MOTION -------------------------------------------------------------------------------------------------------
def EOM_body(m,n,dof):
    l = ('body',1)
    if dof == 'x':
        S = BW/m.m[l] # scale factor
        return m.ddX[n,l,dof] == -S*m.F_c[n,'x']
    
    if dof == 'y':
        S = BW/m.m[l]
        return m.ddX[n,l,dof] == -S*m.F_c[n,'y'] - m.g
    
    if dof == 'theta':
        # constraint forces act through the COM here so they don't create moments
        S = BW
        tau_hip = -BW*m.tau_a[n] - BW*(m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
        return m.ddX[n,l,dof] == tau_hip/m.In[l]

m.EOM_body = Constraint(m.N, m.DOF, rule = EOM_body)

def EOM_leg1(m,n,dof):
    l = ('leg',1)
    if dof == 'x':
        S = BW/m.m[l]
        F_knee = -S*(m.F_a[n]+m.F_r[n,'ps']-m.F_r[n,'ng'])*sin(m.X[n,l,'theta'])
        return m.ddX[n,l,dof] == S*m.F_c[n,'x'] + F_knee
    
    if dof == 'y':
        S = BW/m.m[l]
        F_knee = S*(m.F_a[n]+m.F_r[n,'ps']-m.F_r[n,'ng'])*cos(m.X[n,l,'theta'])
        return m.ddX[n,l,dof] == S*m.F_c[n,'y'] + F_knee - m.g
    
    if dof == 'theta':
        # the parallel forces don't create moments, so you only need to consider the constraint forces
        # ...but the knee constraint forces act through the COM so they don't create moments either
        S1 = BW # scale factor for forces
        S2 = BW # scale factor for torques
        mo_hip = -S1*m.F_c[n,'x']*0.5*m.len[l]*cos(m.X[n,l,'theta']) - S1*m.F_c[n,'y']*0.5*m.len[l]*sin(m.X[n,l,'theta'])
        tau_hip = S2*m.tau_a[n] + S2*(m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
        tau_knee = -S2*m.tau_c[n]
        return m.In[l]*m.ddX[n,l,dof] == mo_hip + tau_hip + tau_knee

m.EOM_leg1 = Constraint(m.N, m.DOF, rule = EOM_leg1)

def EOM_leg2(m,n,dof):
    l = ('leg',2)
    if dof == 'x':
        S = BW/m.m[l]
        F_knee = S*(m.F_a[n]+m.F_r[n,'ps']-m.F_r[n,'ng'])*sin(m.X[n,l,'theta'])
        return m.ddX[n,l,dof] == S*(m.GRF[n,'x','ps'] - m.GRF[n,'x','ng']) + F_knee
    
    if dof == 'y':
        S = BW/m.m[l]
        F_knee = -S*(m.F_a[n]+m.F_r[n,'ps']-m.F_r[n,'ng'])*cos(m.X[n,l,'theta'])
        return m.ddX[n,l,dof] == S*m.GRF[n,'y','ps'] + F_knee - m.g
    
    if dof == 'theta':
        S1 = BW
        S2 = BW
        mo_GRF = -S1*(m.GRF[n,'x','ps']-m.GRF[n,'x','ng'])*0.5*m.len[l]*cos(m.X[n,l,'theta']) - S1*m.GRF[n,'y','ps']*0.5*m.len[l]*sin(m.X[n,l,'theta'])
        tau_knee = S2*m.tau_c[n]
        return m.In[l]*m.ddX[n,l,dof] == mo_GRF + tau_knee
        
m.EOM_leg2 = Constraint(m.N, m.DOF, rule = EOM_leg2)       

## Hannah Montana initialization

Random guesses, especially when the values vary over a wide range, are extremely difficult to get a solution out of. You never want to go full **\*holds up spork\*** . I often employ something I'm now unfortunately stuck calling the *Hannah Montana approach* after I called it that as a joke one time. By which I mean ~~the model starts off as a sweet, wholesome Disney star, proceeds to ineffectual twerking and at some point produces a surprisingly pleasing psychedelic pop album with the help of The Flaming Lips~~ it's (maybe) the best of both worlds. What this involves is letting the values vary randomly, but in a smaller range around where I more or less expect them to actually be. So I create a guide trajectory (here I'm using linearly-spaced values from 0 to 5 metres as a guide for $x$) and then add random noise to that.

In [ ]:
D = 5.0 # final distance

guide = np.linspace(0.0,D,N)

for n in range(1,N+1):
    m.X[n,('body',1),'x'].value = guide[n-1]+np.random.uniform(-0.25,0.25)
    m.X[n,('body',1),'y'].value = np.random.uniform(0.5,1)
    m.X[n,('body',1),'theta'].value = np.random.uniform(-0.1,0.1)
    m.th_joint[n].value = np.random.uniform(-np.pi/6,np.pi/6)
    m.r_joint[n].value = np.random.uniform(0.0,m.X[n,('body',1),'y'].value-0.5*m.len[('leg',1)])
            

## Sanity checks
One of the tricky and often frustrating things about trajectory optimization is that you have to code everything before you can test anything. Due to the aforementioned interdependency issue, it can be very difficult to pinpoint the exact cause of strange behaviour or isolate any one part to check if it's working correctly.

To help with this, there are a few simple tests I like to run right after finishing a model to help find and kill some of the more obvious bugs. Because the cost function can potentially lead the solver into a difficult place, I'd recommend running these with a *feasibility* objective: that is, no other goal besides satisfying the penalty. 

### The high drop test
**How you do it:** Set the initial condition to rest in a known pose (say, all angles = 0) high enough above the ground that it can fall without hitting it, fix all actuator forces/ torques to zero and then just let it drop.

**What's the point?** This test will reveal a few things:
1. Big, stupid problems: if it comes back infeasible, your model is broken somewhere. Time to make yourself some coffee, dust off the ol' `m.pprint()` statement and try to find that one equation you copied and forgot to change a variable name in.
2. Ground interaction problems: if a ground reaction force is anything other than zero when it's three metres in the air, something's wrong.
3. Equations of motion problems: check `m.ddX.pprint()`. You want to see zero acceleration everywhere, and -9.81 in the $y$ direction. If you see anything else, something's wrong. This is especially important when you're working in system space coordinates, where you have all those tricky constraint forces to balance; you have to make sure they're not somehow conspiring to create motion where none should happen (this is why we set the actuator actions to zero).

### The low drop test
**How you do it:** Same as the high drop, but now you're starting with the foot only a short way above the ground (e.g. 1 cm) so it's guaranteed to land.

**What's the point?** To check if your ground contacts work to activate the ground reaction forces and successfully stop the feet.

### The standing test
**How you do it:** Deactivate the actuators as in the other tests, and start the model at rest in a known pose with its feet (well, foot in this case) on the ground. It might also be worth trying a few postures that push the limits of the joints e.g. have the prismatic knee joint compressed all the way in.

**What's the point?** While the low drop is meant to check if your contacts behave as expected in impulsive collisions, the standing test checks whether they can handle sustained contact.

### The hop test
**How you do it:** Now we get to add some power :) Start the model at rest in contact with the ground, and set the final condition to some $x$ or $y$ that will either force it to move forward or jump in the air. 

**What's the point?** To see if the actuator force limits you've set allow enough *oomph* to get the dude off the ground. If you're brave and want to test the limits in the other direction, you can tell it to maximize the final $x$ or $y$ position (by making $-x[N]$ or $-y[N]$ your objective) to make sure it can't shoot itself into orbit, either.

**Note:** To save on time, use as few nodes as possible to do these tests. Especially for something like the high drop, you should be able to see what you're trying to see with just $N = 10$.

In [ ]:
# # HIGH DROP -----------------------------------------------------------------------------------------------------------------

# # initial condition
# m.footp[1,'y'].value = 2.0
# m.X[1,('body',1),'x'].value = 0
# m.X[1,('body',1),'theta'].value = 0
# m.X[1,('leg',1),'theta'].value = 0
# m.X[1,('leg',2),'theta'].value = 0
# m.r_joint[2].value = 0.0

# m.footp[1,'y'].fixed = True
# m.X[1,('body',1),'x'].fixed = True
# m.X[1,('body',1),'theta'].fixed = True
# m.X[1,('leg',1),'theta'].fixed = True
# m.X[1,('leg',2),'theta'].fixed = True
# m.r_joint[2].fixed = True

# for dof in DOFs:  
#     for l in links:
#         m.dX[1,l,dof].value = 0
#         m.dX[1,l,dof].fixed = True
    
#     for n in range(1,N+1):
#         m.tau_a[n].value = 0
#         m.tau_a[n].fixed = True
#         m.F_a[n].value = 0
#         m.F_a[n].fixed = True     

In [ ]:
# # LOW DROP -----------------------------------------------------------------------------------------------------------------

# # initial condition
# m.footp[1,'y'].value = 0.1
# m.X[1,('body',1),'x'].value = 0
# m.X[1,('body',1),'theta'].value = 0
# m.X[1,('leg',1),'theta'].value = 0
# m.X[1,('leg',2),'theta'].value = 0
# m.r_joint[2].value = 0.5

# m.footp[1,'y'].fixed = True
# m.X[1,('body',1),'x'].fixed = True
# m.X[1,('body',1),'theta'].fixed = True
# m.X[1,('leg',1),'theta'].fixed = True
# m.X[1,('leg',2),'theta'].fixed = True
# m.r_joint[2].fixed = True

# for dof in DOFs:  
#     for l in links:
#         m.dX[1,l,dof].value = 0
#         m.dX[1,l,dof].fixed = True
    
#     for n in range(1,N+1):
#         m.tau_a[n].value = 0
#         m.tau_a[n].fixed = True
#         m.F_a[n].value = 0
#         m.F_a[n].fixed = True     

In [ ]:
# # STANDING ------------------------------------------------------------------------------------------------------------------

# # initial condition
# m.footp[1,'y'].value = 0.0
# m.X[1,('body',1),'x'].value = 0
# m.X[1,('body',1),'theta'].value = 0
# m.X[1,('leg',1),'theta'].value = 0
# m.X[1,('leg',2),'theta'].value = 0
# m.r_joint[2].value = 0.0

# m.footp[1,'y'].fixed = True
# m.X[1,('body',1),'x'].fixed = True
# m.X[1,('body',1),'theta'].fixed = True
# m.X[1,('leg',1),'theta'].fixed = True
# m.X[1,('leg',2),'theta'].fixed = True
# m.r_joint[2].fixed = True

# for dof in DOFs:  
#     for l in links:
#         m.dX[1,l,dof].value = 0
#         m.dX[1,l,dof].fixed = True
    
#     for n in range(1,N+1):
#         m.tau_a[n].value = 0
#         m.tau_a[n].fixed = True
#         m.F_a[n].value = 0
#         m.F_a[n].fixed = True    

In [ ]:
# # HOP -----------------------------------------------------------------------------------------------------------------------

# initial condition
# m.footp[1,'y'].value = 0.0
# m.X[1,('body',1),'x'].value = 0
# m.X[1,('body',1),'theta'].value = 0
# m.X[1,('leg',1),'theta'].value = 0
# m.X[1,('leg',2),'theta'].value = 0

# m.footp[1,'y'].fixed = True
# m.X[1,('body',1),'x'].fixed = True
# m.X[1,('body',1),'theta'].fixed = True
# m.X[1,('leg',1),'theta'].fixed = True
# m.X[1,('leg',2),'theta'].fixed = True

# for dof in DOFs:  
#     for l in links:
#         m.dX[1,l,dof].value = 0
#         m.dX[1,l,dof].fixed = True
            
# # final condition
# m.X[N,('body',1),'y'].setlb(1.2)

In [ ]:
#SPRINT --------------------------------------------------------------------------------------------------------------------
#sprint 5m from rest

#initial condition
m.footp[1,'y'].value = 0.0
m.X[1,('body',1),'x'].value = 0.0
m.X[1,('body',1),'theta'].value = 0.0
m.X[1,('leg',1),'theta'].value = 0.0
m.X[1,('leg',2),'theta'].value = 0.0

m.footp[1,'y'].fixed = True
m.X[1,('body',1),'x'].fixed = True
m.X[1,('body',1),'theta'].fixed = True
m.X[1,('leg',1),'theta'].fixed = True
m.X[1,('leg',2),'theta'].fixed = True

for dof in DOFs:  
    for l in links:
        m.dX[1,l,dof].value = 0.0
        m.dX[1,l,dof].fixed = True
            
# final condition
m.X[N,('body',1),'x'].setlb(5.0)


In [ ]:
# solving
#opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 600 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True) 

In [ ]:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

penalty_sum = 0
for n in range(1,N+1):
    for gc in ground_constraints:
        penalty_sum += m.ground_penalty[n,gc].value
    for jc in joint_constraints:
        for j in joints:
            penalty_sum += m.joint_penalty[n,j,jc].value

print penalty_sum

#m.pprint() 

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,10])
    ax.set_ylim([0,3])
    
    #plot body
    body_xb = m.X[i,('body',1),'x'].value - 0.5*m.len[('body',1)]*cos(m.X[i,('body',1),'theta'].value)
    body_yb = m.X[i,('body',1),'y'].value - 0.5*m.len[('body',1)]*sin(m.X[i,('body',1),'theta'].value)
    body_xf = m.X[i,('body',1),'x'].value + 0.5*m.len[('body',1)]*cos(m.X[i,('body',1),'theta'].value)
    body_yf = m.X[i,('body',1),'y'].value + 0.5*m.len[('body',1)]*sin(m.X[i,('body',1),'theta'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    leg1_xt = m.X[i,('leg',1),'x'].value - 0.5*m.len[('leg',1)]*sin(m.X[i,('leg',1),'theta'].value)
    leg1_yt = m.X[i,('leg',1),'y'].value + 0.5*m.len[('leg',1)]*cos(m.X[i,('leg',1),'theta'].value)
    leg1_xb = m.X[i,('leg',1),'x'].value + 0.5*m.len[('leg',1)]*sin(m.X[i,('leg',1),'theta'].value)
    leg1_yb = m.X[i,('leg',1),'y'].value - 0.5*m.len[('leg',1)]*cos(m.X[i,('leg',1),'theta'].value)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:black')
    
    #plot leg 2
    leg2_xt = m.X[i,('leg',2),'x'].value - 0.5*m.len[('leg',2)]*sin(m.X[i,('leg',2),'theta'].value)
    leg2_yt = m.X[i,('leg',2),'y'].value + 0.5*m.len[('leg',2)]*cos(m.X[i,('leg',2),'theta'].value)
    leg2_xb = m.X[i,('leg',2),'x'].value + 0.5*m.len[('leg',2)]*sin(m.X[i,('leg',2),'theta'].value)
    leg2_yb = m.X[i,('leg',2),'y'].value - 0.5*m.len[('leg',2)]*cos(m.X[i,('leg',2),'theta'].value)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')
    
update = lambda i: plot_robot(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
